In [1]:
import rtree
import geopandas as gpd
import numpy as np
import os, glob

/home/napo/.local/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.1-CAPI-1.13.3) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
comuni_trentini = gpd.read_file('data' + os.sep + 'comuni_trentini.gpkg')

In [3]:
comuni_trentini = comuni_trentini.to_crs(epsg=4326)

In [4]:
confini_pat = comuni_trentini.dissolve()
confini_pat.to_file("confini_pat.geojson",driver="GeoJSON")

In [5]:
confine_trento = comuni_trentini[comuni_trentini.PRO_COM_T=='022205']
confine_trento.to_file("trento.geojson",driver="GeoJSON")

In [6]:
comuni5k  = comuni_trentini[comuni_trentini.POPOLAZIONE <= 5000]

In [7]:
os.chdir('data')
geojson_files = glob.glob("*.geojson")
os.chdir("..")
areasdata = "data" + os.sep + "areas30km"
if not os.path.exists(areasdata):
    os.makedirs(areasdata)

In [8]:
confine_verona = gpd.read_file('data' + os.sep + "verona.geojson")
confine_belluno = gpd.read_file('data' + os.sep + "belluno.geojson")
confine_bolzano = gpd.read_file('data' + os.sep + 'bolzano.geojson')
exclude = []
exclude.append("verona")
exclude.append("belluno")
exclude.append("bolzano")

DriverError: data/verona.geojson: No such file or directory

In [47]:
def getGeometry(comune_id):
    geocomune = comuni5k[comuni5k.PRO_COM_T == comune_id]
    geocomune30k = gpd.read_file('data' + os.sep + comune_id + ".geojson")
    unionecomune30k = gpd.overlay(geocomune30k, geocomune, how='union').dissolve()
    unionecomune30k['CROSSTN'] = 0
    unionecomune30k['OUTSIDE'] = 0
    unionecomune30k['CROSSBL'] = 0
    unionecomune30k['CROSSBZ'] = 0
    area = unionecomune30k.geometry[unionecomune30k.index[0]]
    trento = confine_trento.geometry[confine_trento.index[0]]
    verona = confine_verona.geometry[confine_verona.index[0]]
    belluno = confine_belluno.geometry[confine_belluno.index[0]]
    bolzano = confine_bolzano.geometry[confine_bolzano.index[0]]
    provincia = confini_pat.geometry[confini_pat.index[0]]
    if (area.intersects(trento)):
        unionecomune30k = gpd.overlay(unionecomune30k,confine_trento,how='difference')
        unionecomune30k['CROSSTN'] = 1
    if (area.intersects(verona)):
        unionecomune30k = gpd.overlay(unionecomune30k,confine_verona,how='difference')
        unionecomune30k['CROSSVR'] = 1
    if (area.intersects(belluno)):
        unionecomune30k = gpd.overlay(unionecomune30k,confine_belluno,how="difference")
        unionecomune30k['CROSSBL'] = 1
    if (area.intersects(bolzano)):
        unionecomune30k = gpd.overlay(unionecomune30k,confine_bolzano,how="difference")
        unionecomune30k['CROSSBZ'] = 1
    if (area.intersects(provincia)):
        unionecomune30k['OUTSIDE'] = 1
    return(unionecomune30k[['geometry','COMUNE','PRO_COM_T','CROSSTN','CROSSBL','CROSSBZ']])

In [53]:
for geojson in geojson_files:
    comune_id = geojson.replace(".geojson","")
    if ((comune_id in exclude) == False):
        filesize = os.stat("data" + os.sep + geojson).st_size
        if filesize > 194:
            newgeom = getGeometry(comune_id)
            filename = areasdata + os.sep + comune_id + ".geojson"
            newgeom.to_file(filename,driver="GeoJSON")
        else:
            print(comune_id)

022182
022183
022180


In [54]:
print(comune_id)

022124


In [55]:
newgeom.plot()

,geometry,COMUNE,PRO_COM_T,CROSSTN,CROSSBL,CROSSBZ
0,"MULTIPOLYGON (((10.89726 45.83042, 10.89680 45...",Nago-Torbole,022124,1,0,0


In [76]:
a

,geometry,COMUNE,PRO_COM_T,CROSSTN
0,"MULTIPOLYGON (((10.83093 46.05141, 10.82963 46...",Terre d'Adige,022251,1


In [43]:
a[['geometry','COMUNE','PRO_COM_TN','CROSSTN']].to_file("/tmp/ciapa.shp")

In [28]:
union.to_file("union.geojson",driver="GeoJSON")